In [5]:
from random import random
class RandomHash:
    def __init__(self):
        self.table = dict()
    def of(self, x):
        if x not in self.table:
            self.table[x] = random()
        return self.table[x]

In [6]:
def build_array(s, q):
    """
    build an array A with |s | − q + 1 elements, where the i-th element
    A[i] = F (si..i+q−1), where F : Σq → (0, 1) is a random hash function
    """
    from random import random
    F = RandomHash()
    A = []
    for i in range(len(s)-q+1):
        A.append(F.of(s[i:i+q]))
    return A    

In [7]:
build_array('amigo', 3)

[0.598152218891704, 0.7818686822714424, 0.12328346654685007]

In [10]:
def Rank(s, r, q):
    """
    Input: input string s; minimum rank r
    Output: rank array R = {(pi, Ri )}: pi denotes the index of the letter in
    s corresponding to the i-th pair, and Ri is its rank
    """
    A = build_array(s, q)
    R = [(0, float('inf'))] # the first character of s has ranking infinity
    for i in range(len(A)):
        x = 1
        while i-x >= 0 and i+x < len(A):
            if A[i] < min(A[i+x], A[i-x]):
                x += 1
            else:
                break
        if x > r:
            R.append((i, x-1))
    R.append((len(s)-q, float('inf')))
    return R   

In [40]:
Rank('ACGTTCGACTGGTTAG', 0, 3)

[(0, inf),
 (0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 6),
 (8, 0),
 (9, 0),
 (10, 0),
 (11, 2),
 (12, 0),
 (13, 0),
 (13, inf)]

In [57]:
def Partition(s, R, start, end):
    """
    Input: input string s; rank array R = {(pi, Ri )} of s; two indices start and end
    Output: set of partitions P = {(ssub , l)}, where (ssub , l) denotes a
    substring ssub with level l
    """
    if end <= start + 1:
        return []
    maxR = float('-inf')
    M = []
    i = start + 1
    while i < end:
        if R[i][1] > maxR:
            maxR = R[i][1]
            M = [i]
        if R[i][1] == maxR:
            M.append(i)
        i += 1
    P = []
    M = [start] + M + [end]
    M.sort()
    for j in range(len(M)-1):
        u = M[j]
        v = M[j+1]
        pu = R[u][0]
        pv_minus_1 = R[v-1][0]
        pv = R[v][0]
        P.append((s[pu:pv_minus_1 + 1], min(R[u][1], R[v][1])))
        P = P+(Partition(s, R, u, v))
    return P

In [58]:
R = Rank('ACGTTCGACTGGTTAG', 0, 3)
Partition('ACGTTCGACTGGTTAG', R, 0, len(R)-1)

[('ACGT', 4),
 ('AC', 1),
 ('A', 0),
 ('A', 0),
 ('A', 0),
 ('C', 0),
 ('', 1),
 ('GT', 1),
 ('G', 0),
 ('', 0),
 ('T', 0),
 ('', 4),
 ('TCGACTGGTT', 4),
 ('TC', 1),
 ('T', 0),
 ('', 0),
 ('C', 0),
 ('', 1),
 ('GA', 1),
 ('G', 0),
 ('', 0),
 ('A', 0),
 ('CT', 1),
 ('C', 0),
 ('', 0),
 ('T', 0),
 ('GG', 1),
 ('G', 0),
 ('', 0),
 ('G', 0),
 ('TT', 1),
 ('T', 0),
 ('', 0),
 ('T', 0)]

In [72]:
from random import randint
from collections import defaultdict
class RandomHashInt:
    def __init__(self):
        self.table = dict()
    def of(self, x):
        if x not in self.table:
            self.table[x] = randint(0,1000)
        return self.table[x]
class HashTablesList:
    def __init__(self):
        self.tables = {}
    def get(self, l):
        if l not in self.tables:
            return None
        return self.tables[l]
    def insert(self, l):
        self.tables[l] = {'table': defaultdict(set), 'random_function': RandomHashInt() }

In [88]:
def build_index(S, q):
    """
    Input: set of input strings S = {s1, . . . , sn }
    Output: set of hash tables {(Hi, fi )}, where for the i-th hash table, each
    string s is hashed into the fi(s)-th bucket of Hi
    """
    hash_tables = HashTablesList()
    for i,si in enumerate(S):
        Ri = Rank(si, 0, q)
        Pi = Partition(si, Ri, 0, len(Ri)-1)
        for (ssub, l) in Pi:
            if hash_tables.get(l) is None:
                hash_tables.insert(l)
            fl = hash_tables.get(l)['random_function']
            hash_tables.get(l)['table'][(ssub, fl.of(ssub))].add(i)
    return hash_tables

In [91]:
S = ['ACGTTCGACTGGTTAG',
     'CCGTTCGAACTGGTTAG',
     'ACATTCGACTGGTTGAG',
     'TCGAACGTTCGAACGT']
index = build_index(S, 3)

In [92]:
index.tables

{6: {'table': defaultdict(set,
              {('ACGTTC', 437): {0}, ('', 953): {0}, ('GACTGGTT', 994): {0}}),
  'random_function': <__main__.RandomHashInt at 0x7f4eb46445b0>},
 2: {'table': defaultdict(set,
              {('AC', 435): {0},
               ('', 271): {0, 1},
               ('GTTC', 625): {0},
               ('GACTG', 778): {0},
               ('GTT', 529): {0},
               ('TTC', 721): {1},
               ('GAAC', 757): {1}}),
  'random_function': <__main__.RandomHashInt at 0x7f4eb462d580>},
 0: {'table': defaultdict(set,
              {('A', 52): {0, 1, 2, 3},
               ('C', 654): {0, 1, 2, 3},
               ('G', 198): {0, 1, 2, 3},
               ('', 798): {0, 1, 2, 3},
               ('T', 237): {0, 1, 2, 3}}),
  'random_function': <__main__.RandomHashInt at 0x7f4eb462ddf0>},
 1: {'table': defaultdict(set,
              {('GA', 7): {0, 1},
               ('', 276): {0, 1, 2, 3},
               ('CTG', 516): {0},
               ('AC', 831): {1, 2},
       

In [93]:
def ED(s1, s2):
    # Edit distance between two strings. Levenshtein distance, in this case
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [94]:
ED('abc', 'cba')

2

In [ ]:
def min_search_threshold(S, t, K):
    """
    Input: set of strings S = {s1, . . . , sn }; query string t; distance threshold K
    Output: O ← {i | si ∈ S; ED(si, t) ≤ K }
    """